In [63]:
#Dependencies
import glob
import numpy as np
import pandas as pd
import seaborn as sb

from pylab import rcParams

In [64]:
#Set dimensions for plots
%matplotlib inline
rcParams["figure.figsize"]=5,4
sb.set_style("whitegrid")

In [104]:
def defense(*games):
    #Combine all xlsx files into 1 Dataframe
    all_data = pd.DataFrame()
    for f in glob.glob(r"Becker*.xlsx"):
        df = pd.read_excel(f)
        all_data = all_data.append(df, ignore_index=True)
    all_games = all_data[['A/B/0', 'BACK LEVEL', 'BACK SET', 'BLITZ', 'CHECK COV.', 'COMMENT',
       'COVERAGE', 'D/D GROUPS', 'DEF FRONT', 'DIST', 'DN', 'DPT', 'FIB',
       'FIELD/BND', 'FORM TYPE', 'FZ', 'GN/LS', 'HASH', 'ODK', 'OFF FORM',
       'OFF MO', 'OFF PLAY', 'PASS CONCEPTS', 'PERIOD', 'PERSONNEL', 'PLAY #',
       'PLAY DIR', 'PLAY TYPE', 'PROT.', 'QB DIRECTION', 'RESULT', 'ROUTES',
       'RUN SCH', 'RUN STR', 'TITLE', 'WR SPLITS', 'YARD LN']]
    
    #Subset for creating EFFECTIVE Column
    table = all_games.dropna(subset=["OFF FORM"])
    table["DD"] = table[["DN", "DIST"]].astype(str).sum(axis=1)
    table["playNum"] = table["PLAY #"]
    table["DOWN"] = table["DN"]
    table["ydLine"] = table["YARD LN"]
    table = table[["PLAY #","DN","playNum","ydLine","DOWN",
                   "DIST","DD","GN/LS","PLAY TYPE", "OFF FORM", 
                   "OFF PLAY"]].set_index(["PLAY #", "DN"])
    table.fillna(value = table["GN/LS"].median(), inplace=True)
    conditions = [
    ((table["GN/LS"] >=4) & (table["DD"] == "1.010.0")), 
    (table["GN/LS"] >= (table["DIST"]/2)), 
    (table["GN/LS"] < (table["DIST"]/2))
    ]
    choices = ["Yes", "Yes", "No"]
    table["EFFECTIVE"] = np.select(conditions, choices)
    eff = pd.crosstab([table["OFF FORM"], table["OFF PLAY"]],
                      table["EFFECTIVE"])
#     return eff
    eff.to_csv("Becker_Effective.csv")
    
    #Down Distance Playcalling
    DD = pd.crosstab([all_data["DN"], 
                     all_data["DIST"]], 
                    [all_data["OFF FORM"], 
                     all_data["OFF PLAY"]])
#     return DD
    DD.to_csv("Becker_DD.csv")
    
    #Offensive Playcall Prediction
    all_data["PREVIOUS PLAY"] = all_data["OFF PLAY"].shift(1)
    all_data["prevPlay"] = all_data["PREVIOUS PLAY"]
    all_data["nextPlay"] = all_data["OFF PLAY"]
    prediction = pd.crosstab(all_data["prevPlay"], all_data["nextPlay"])
#     return prediction
    prediction.to_csv("Becker_Prediction.csv")
    
    #Scenarios
    ddConditions = [
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 3.0)),
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 2.0)),
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 4.0) & (table["DIST"].shift(1) <= 3.0)), 
    ((table["GN/LS"] > 0) & (table["DOWN"] == 1.0) & (table["DOWN"].shift(1) == 4.0) & (table["DIST"].shift(1) >= 4.0))
    ]
    ddChoices = ["3rd Down Conversion", "2nd Down Conversion", 
               "4th and Short Conversion", "4th and Long Conversion"]
    table["Down/Dist Scenarios"] = np.select(ddConditions, ddChoices)
    
    #Game Changing Scenarios
    gcConditions = [
    (table["GN/LS"] > 30)
    ]
    gcChoices = ["Run/Pass > 30 yds"]
    table["Game Changing Scenarios"] = np.select(gcConditions, gcChoices)
    
    #Run/Pass Scenarios
    rpConditions = [
    ((table["GN/LS"] > 0) & (table["DD"] == "1.010.0")), 
    (table["GN/LS"] < 0), 
    (table["GN/LS"] < 5), 
    (table["GN/LS"] > 10), 
    ((table["GN/LS"] > 0) & (table["PLAY TYPE"] == "Pass")),
    ((table["GN/LS"] > 0) & (table["PLAY TYPE"] == "NaN")),
    ((table["GN/LS"] > 10) & (table["PLAY TYPE"] == "Pass"))
    ]
    rpChoices = ["Run for 1st Down", "Run < 0 yds", "Run < 5 yds", "Run < 10 yds", 
                  "Pass Complete", "Pass Incomplete", "Pass > 10 yds"]
    table["Run/Pass Scenarios"] = np.select(rpConditions, rpChoices)
    
    scenarios = pd.crosstab([table["OFF FORM"], 
                             table["OFF PLAY"]], 
                            [table["Down/Dist Scenarios"], 
                             table["Game Changing Scenarios"], 
                             table["Run/Pass Scenarios"]])
#     return scenarios
    scenarios.to_csv("Becker_Scenarios.csv")
    #Formation
    form = pd.crosstab(all_games["OFF FORM"], 
                 all_games["OFF PLAY"])
#     return form
    form.to_csv("Becker_Formations.csv")
    
    #Fieldzones
    columns = all_data[["OFF PLAY", "OFF FORM", "FZ"]]
    fzForm = pd.crosstab([columns["FZ"], columns["OFF FORM"]], 
                            columns["OFF PLAY"])
#     return fzForm
    fzForm.to_csv("Becker_FZ_Plays.csv")
    
    #DPT
    dpt_data = all_data.dropna(subset=["OFF PLAY"])
    dpt_columns = dpt_data[["D/D GROUPS", "DPT", "FZ", "PERSONNEL"]]
    dptDD = pd.crosstab(dpt_columns["D/D GROUPS"], dpt_columns["DPT"])
#     return dptDD
    dptDD.to_csv("Becker_dptDD.csv")
    dptFZ = pd.crosstab(dpt_columns["FZ"], dpt_columns["DPT"])
#     return dptFZ
    dptFZ.to_csv("Becker_dptFZ.csv")
    dptPER = pd.crosstab(dpt_columns["PERSONNEL"], dpt_columns["DPT"])
#     return dptPER
    dptPER.to_csv("Becker_dptPER.csv")
    
    #FIB
    fib_Data = all_data.dropna(subset=["FIB"])
    columns = fib_Data[["OFF PLAY", "OFF FORM"]]
    persTable = pd.crosstab(columns["OFF PLAY"], columns["OFF FORM"])
#     return persTable
    persTable.to_csv("Becker_FIB_Playcalling.csv")
    
    fzFIB = pd.crosstab([fib_Data["FZ"], fib_Data["DPT"], 
                         fib_Data["OFF FORM"]], 
                         fib_Data["OFF PLAY"])
#     return fzFIB
    fzFIB.to_csv("Becker_FIB_FZ_PlayType.csv")
    
    #Pass Defense
    #Pass Protections
    pass_data = all_data.dropna(subset=["PLAY TYPE"])
    pDEF = pass_data[["PERSONNEL", "OFF FORM", 
                        "BACK LEVEL", "BACK SET", 
                        "OFF PLAY", "PASS CONCEPTS", 
                        "PROT.", "FORM TYPE", 
                        "OFF MO", "RESULT", "GN/LS", "PLAY TYPE"]]
    pPro = pd.crosstab([pDEF["PERSONNEL"], pDEF["OFF FORM"],
                                pDEF["BACK SET"]], 
                        pDEF["PROT."])
#     return pPro
    pPro.to_csv("Becker_pPro.csv")
    
    #Pass Routes
    pRoutes = pd.crosstab([pDEF["PERSONNEL"], pDEF["OFF FORM"],
                                    pDEF["BACK SET"]], 
                            pDEF["PASS CONCEPTS"])
#     return pRoutes
    pRoutes.to_csv("Becker_pRoutes.csv")
    #FZ, DN, DIST, PASSING
    passSit = all_data.dropna(subset=["ROUTES"])
    passColumns = passSit[["PERSONNEL", "OFF FORM", 
                        "BACK LEVEL", "BACK SET", 
                        "OFF PLAY", "PASS CONCEPTS", 
                        "PROT.", "FORM TYPE", 
                        "OFF MO", "RESULT", "GN/LS", 
                        "PLAY TYPE", "ROUTES", "FZ", "DN", "DIST"]]
    passTable = pd.crosstab([passColumns["FZ"], 
                             passColumns["DN"], passColumns["DIST"]],
                            [passColumns["OFF FORM"], 
                            passColumns["OFF PLAY"]])
#     return passTable
    passTable.to_csv("Becker_FZ_Passing.csv")

defense()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [ ]:
#Breakdown List:
#eff.to_csv("Becker_Effective.csv")
#DD.to_csv("Becker_DD.csv")
#prediction.to_csv("Becker_Prediction.csv")
#scenarios.to_csv("Becker_Scenarios.csv")
#form.to_csv("Becker_Formations.csv")
#fzForm.to_csv("Becker_FZ_Plays.csv")
#dptDD.to_csv("Becker_dptDD.csv")
#dptFZ.to_csv("Becker_dptFZ.csv")
#dptPER.to_csv("Becker_dptPER.csv")
#persTable.to_csv("Becker_FIB_Playcalling.csv")
#fzFIB.to_csv("Becker_FIB_FZ_PlayType.csv")
#pPro.to_csv("Becker_pPro.csv")
#pRoutes.to_csv("Becker_pRoutes.csv")
#passTable.to_csv("Becker_FZ_Passing.csv")